In [6]:
from google.colab import drive
drive.mount('/content/drive/')

import pandas as pd
import numpy as np
import shutil
import os
import torch
from transformers import BertTokenizer, BertModel
from tqdm import tqdm

# Check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)  # Move model to GPU

def get_bert_embeddings(text):
    # Tokenize and get input IDs and attention masks
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)

    # Move inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Get embeddings from BERT
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract last hidden state
    last_hidden_state = outputs.last_hidden_state

    return last_hidden_state

def max_pooling(embeddings):
    # Max pooling along the sequence dimension
    return torch.max(embeddings, dim=1).values

# Define the path to the folder containing CSV files
folder_path = '/content/drive/MyDrive/MiB Dataset/NPY Files/split files/subfolder_1/'

# List all CSV files in the folder
files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialize dictionary to store user-wise pooled outputs
user_pooled_outputs = {}

# Wrap the file processing loop with tqdm to show progress for each file
for file_name in tqdm(files, desc="Processing CSV files"):
    file_path = os.path.join(folder_path, file_name)
    print(f"Processing file: {file_path}")

    try:
        # Load the CSV file
        df = pd.read_csv(file_path)

        # Extract the user ID and text columns
        user_ids = df['user_id'].tolist()  # Replace 'user_id' with the actual column name
        texts = df['text'].tolist()  # Replace 'text' with the actual column name

        # Process each text entry
        for user_id, text in zip(user_ids, texts):
            try:
                embeddings = get_bert_embeddings(text)
                pooled_output = max_pooling(embeddings)
                pooled_output = pooled_output.cpu()  # Move pooled output to CPU

                # Add pooled output to the corresponding user's list
                if user_id not in user_pooled_outputs:
                    user_pooled_outputs[user_id] = []
                user_pooled_outputs[user_id].append(pooled_output)

            except Exception as e:
                print(f"Error processing text: {text[:30]}... Error: {e}")

    except Exception as e:
        print(f"Error processing file {file_name}: {e}")

# Compute the final pooled output for each user
final_user_embeddings = []

for user_id, outputs in user_pooled_outputs.items():
    if outputs:
        # Stack and aggregate the pooled outputs for each user
        pooled_outputs_tensor = torch.stack(outputs)
        user_embedding = torch.max(pooled_outputs_tensor, dim=0)  # Average pooling across tweets
        final_user_embeddings.append(user_embedding)

# Convert final embeddings to a tensor
if final_user_embeddings:
    final_user_embeddings_tensor = torch.stack(final_user_embeddings)
    print(f"Shape of final user embeddings: {final_user_embeddings_tensor.shape}")

    # Example tensor with shape [no_of_users, 768]
    print(final_user_embeddings_tensor.shape)
else:
    print("No user embeddings to aggregate.")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Processing CSV files:   0%|          | 0/286 [00:00<?, ?it/s]

Processing file: /content/drive/MyDrive/MiB Dataset/NPY Files/split files/subfolder_1/user_2378332362.0.csv


Processing CSV files:   0%|          | 1/286 [00:11<52:22, 11.03s/it]

Processing file: /content/drive/MyDrive/MiB Dataset/NPY Files/split files/subfolder_1/user_2378336040.0.csv


...

Processing file: /content/drive/MyDrive/MiB Dataset/NPY Files/split files/subfolder_1/user_2382986905.0.csv


Processing CSV files: 100%|█████████▉| 285/286 [59:52<00:11, 11.76s/it]

Processing file: /content/drive/MyDrive/MiB Dataset/NPY Files/split files/subfolder_1/user_2382987302.0.csv


Processing CSV files: 100%|██████████| 286/286 [1:00:05<00:00, 12.61s/it]

Shape of final user embeddings: torch.Size([286, 1, 768])
torch.Size([286, 1, 768])


In [7]:
# Convert final embeddings to a tensor
if final_user_embeddings:
    # Concatenate the embeddings along the first dimension
    final_user_embeddings_tensor_new = torch.cat(final_user_embeddings, dim=0)

    print(f"Shape of final user embeddings: {final_user_embeddings_tensor_new.shape}")

    # Example tensor with shape [no_of_users, 768]
    print(final_user_embeddings_tensor_new.shape)
else:
    print("No user embeddings to aggregate.")


Shape of final user embeddings: torch.Size([286, 768])
torch.Size([286, 768])
